In [1]:
# ! pip install -e custom-diffusers/

In [2]:
from custom_diffusers import UNet2DConditionModel
from diffusers import DiffusionPipeline, StableDiffusionPipeline
from optimum.exporters.onnx import export
from optimum.onnxruntime import ORTStableDiffusionPipeline
from pathlib import Path
import onnx
import torch

/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download model

In [3]:
TORCH_DIR = Path("./exported-models/torch")

TORCH_DIR.mkdir(parents=True, exist_ok=True)


ONNX_DIR = Path("./exported-models/onnx")

ONNX_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
MODEL_ID = "runwayml/stable-diffusion-v1-5"


In [5]:
# pipe = DiffusionPipeline.from_pretrained(
#     MODEL_ID,
#     torch_dtype=torch.float16,
#     use_safetensors=True,
# )
# pipe = pipe.to("cuda")

# prompt = "a photo of an astronaut riding a horse on mars"
# image = pipe(prompt).images[0]


# pipe.save_pretrained(TORCH_DIR / "stable-diffusion-v1-5")

# Export models to ONNX

In [6]:
SAVE_PATH = ONNX_DIR / "stable-diffusion-v1-5"

if not SAVE_PATH.exists():
    try:

        SAVE_PATH.mkdir(parents=True)
        pipeline = ORTStableDiffusionPipeline.from_pretrained(MODEL_ID, export=True)
        pipeline.save_pretrained(SAVE_PATH.as_posix())
    except Exception as e:
        print(e)
        SAVE_PATH.rmdir()


In [7]:
# pipeline = ORTStableDiffusionPipeline.from_pretrained(
#     SAVE_PATH.as_posix(),
# )

# del pipeline.vae_encoder.session

# prompt = "sailing ship in storm by Leonardo da Vinci"
# image = pipeline(prompt, num_inference_steps=2).images[0]

# pipeline.save_pretrained("./models/onnx/stable-diffusion-v1-5")

In [8]:

# pipe = DiffusionPipeline.from_pretrained(
#     "./torch-stable-diffusion-v1-5",
#     torch_dtype=torch.float16,
#     use_safetensors=True,
# )
# pipe = pipe.to("cuda")

# prompt = "a photo of an astronaut riding a horse on mars"
# image = pipe(prompt).images[0]



# Export Unet

In [9]:
unet = UNet2DConditionModel.from_pretrained("./exported-models/torch/stable-diffusion-v1-5/unet")

In [10]:
timesteps = torch.tensor([1, 2]).reshape(-1, 1).float()

latent_model_input = torch.randn(2, 4, 64, 64)

prompt_embeds = torch.randn(2, 77, 768)

_ = unet(timestep=timesteps, sample=latent_model_input, encoder_hidden_states=prompt_embeds)

In [11]:
timesteps.shape

torch.Size([2, 1])

In [12]:
from custom_diffusers.config import UNetOnnxConfig

In [13]:
onnx_config = UNetOnnxConfig(unet.config)

In [19]:
inputs = onnx_config.generate_dummy_inputs()
for key in inputs:
    print(key, inputs[key].shape)

sample torch.Size([2, 4, 64, 64])
timestep torch.Size([2, 1])
encoder_hidden_states torch.Size([2, 16, 768])


In [15]:
onnx_path = Path("./exported-models/onnx/unet.onnx")

onnx_inputs, onnx_outputs = export(unet, onnx_config, onnx_path, onnx_config.DEFAULT_ONNX_OPSET)

Using framework PyTorch: 2.3.0+cu121
/home/tran.duc.trungb/triton/triton-playground/02.build_tensorrt_from_sd/custom-diffusers/src/custom_diffusers/unet_2d_condition.py:1228: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if dim % default_overall_up_factor != 0:
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/models/downsampling.py:137: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-pa

In [16]:
onnx_config.inputs

{'sample': {0: 'batch_size', 1: 'num_channels', 2: 'height', 3: 'width'},
 'timestep': {0: 'batch_size'},
 'encoder_hidden_states': {0: 'batch_size', 1: 'sequence_length'}}

In [17]:
onnx.checker.check_model(onnx_path.as_posix())

In [18]:
from optimum.exporters.onnx import validate_model_outputs

validate_model_outputs(
    onnx_config, unet, onnx_path, onnx_outputs, onnx_config.ATOL_FOR_VALIDATION
)


Validating ONNX model exported-models/onnx/unet.onnx...
	-[✓] ONNX model output names match reference model (sample)
	- Validating ONNX Model output "sample":
		-[✓] (2, 4, 64, 64) matches (2, 4, 64, 64)
		-[✓] all values close (atol: 0.001)
